In [ ]:
import itertools
import io
import numpy as np
import openai
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
data_pulling = pd.read_csv("generated_data.csv")
data_pulling


In [ ]:
# correlates the n-1 first columns in dataframe values with the nth column
def overall_correlation(df):
    if df.shape[1] < 2:
        raise ValueError("DataFrame must have at least two columns")
    corr_matrix = df.corr()
    last_column = df.columns[-1]
    correlations = corr_matrix[last_column][:-1]
    overall_corr = correlations.mean()

    return overall_corr


In [ ]:
# cleans dataframe by converting objects to numbers, part of parsing.
def convert_strings_to_numbers(df):
    new_df = pd.DataFrame()

    for column in df.columns:
        if df[column].dtype == "object":
            unique_strings = df[column].unique()
            string_to_number = {
                string: i + 1 for i, string in enumerate(unique_strings)
            }
            new_df[column] = df[column].map(string_to_number)
        else:
            new_df[column] = df[column]

    return new_df


In [ ]:
data_pulling["corr"] = None


In [ ]:


i = 0
for h, p in zip(data_pulling["data"], data_pulling["specific_plan"]):

    if not isinstance(input_string, str):
        data_pulling.at[i, "corr"] = 0
        i += 1
        continue

    input_string = input_string.replace("`", "")
    input_string = input_string.strip()
    lines = input_string.split("\n")
    headers = lines[0].split()
    data_lines = []
    for line in lines[1:]:
        data_lines.append(re.split(r"\s{2,}", line.strip()))
    csv_data = io.StringIO()
    csv_data.write(",".join(headers) + "\n")
    for data_line in data_lines:
        csv_data.write(",".join(data_line) + "\n")

    csv_data.seek(0)

    try:
        df = pd.read_csv(csv_data)
        df.fillna(0, inplace=True)
        df = convert_strings_to_numbers(df)

        val = abs(overall_correlation(df))
        if np.isnan(val):
            val = 0
    except pd.errors.ParserError:
        val = 0

    data_pulling.at[i, "corr"] = val
    print(i)
    i += 1


In [ ]:
data_pulling.to_csv("final_data.csv", index=False)
